# Advection-diffusion in 2d 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from ldc_utils import *

In [ ]:
# wersja numpy z indeksowaniem kartezjanskim
nx = 64
ny = 54

Dyf    = 0.1
cs2 = 1.0/3.0
omega = 1. / (Dyf/cs2+0.5) # relaxation parameter 
print(Dyf,omega)
# weights
w  =   np.array([4/9., 1/9.,1/9.,1/9.,1/9., 1/36.,1/36.,1/36.,1/36.])

c = [(0,0), (1,0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, 1), (-1, -1), (1, -1)]
opp = [c.index( (-c_[0],-c_[1]) )  for c_ in c]
c = np.array(c)
# numpy version
#opp = [np.where(np.all((c == -c_),axis=-1))[0][0]  for c_ in c]

obst = np.ones((nx,ny)).astype(np.bool)
obst[1:-1,1:-1] = False 


def f_eq(rho,u,c=c,w=w):
    cu = np.tensordot(c,u,axes=[1,0])
    f = rho * (1 + cu/cs2 + (cu**2)/(2*cs2**2) - np.sum((u**2),axis=0)/(2*cs2) )
    return f*w[:,np.newaxis,np.newaxis]

T_init_l = lambda x:(np.exp( -((x[0] - 23)**2+(x[1] - 13)**2)/20.0 ))
X,Y = np.mgrid[0:nx,0:ny]
rho_init = T_init_l([X,Y])
u_adv = np.zeros((2,nx,ny))
u_adv[0,:,:] = 0.1
u_adv[1,:,:] = 0.2

f = f_eq(rho_init,u_adv,c=c,w=w)

u_t = []
for iteration in range(100):

    rho = f.sum(axis=0)

    fOut = f - omega * (f-f_eq(rho,u_adv))
    
    for i in range(9):  
        fOut[i,obst] = f[opp[i],obst]

    #f_new = np.empty_like(f)
    for i,(k,l) in enumerate(c):
        #k,l = -k,-l
        # f[i,1:-1,1:-1] = fOut[i,1+k:(nx-1)+k,1+l:(ny-1)+l]
        f[i,:,:] = np.roll(np.roll(fOut[i,:,:],k,axis=0),l,axis=1)
    if iteration%1==0:
        u_t.append( rho.copy() )

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets.widgets import IntSlider
from ipywidgets import interact,Layout
style = Layout(width='70%')

f,ax = plt.subplots(figsize=(8,6))
#ax.imshow(T_init_l([X,Y]),origin='lower')
plt_evo = ax.imshow(u_t[1].T,origin='lower',extent=(0,nx,0,ny))
plt_init = ax.contour(X,Y,T_init_l([X,Y]),colors='r')
ax.set_aspect(1)
@interact(ith=IntSlider(min=0,max=len(u_t)-1,layout=style))
def _(ith):
    plt.title('%f@%d'%(np.sum(u_t[ith]),ith))
    plt_evo.set_array(u_t[ith].T)
    print(np.sum(u_t[ith]))

##  FitzHugh–Nagumo


In [ ]:
import sympy
a = .1
b = .4
d = .31
fx = lambda u,v: u-u**3 - v + d
fy = lambda u,v: u-a-b*v

x0,x1 = -2.,2.
y0,y1 = -2.,2.
nx,ny = 32*8,32*8

X,Y = np.mgrid[0:nx,0:ny]
X = x0 + X*(x1-x0)/(nx-1)
Y = y0 + Y*(y1-y0)/(ny-1)

#X,Y = np.meshgrid(np.linspace(x0,x1,nx),np.linspace(y0,y1,ny))


Fx,Fy = fx(X,Y),fy(X,Y)
f,ax = plt.subplots(figsize=(8,6))
ax.streamplot(X.T,Y.T,Fx.T,Fy.T)
ax.contourf(X.T,Y.T,np.sqrt(Fx**2+Fy**2).T,np.linspace(.0,3,10))




In [ ]:
np.max(np.abs(Fx)),np.max(np.abs(Fy))

In [ ]:
%%time 
# wersja numpy z indeksowaniem kartezjanskim
nx = 100
ny = 100

Dyf    = 0.01
u0 = 11.0
cs2 = 1.0/3.0
omega = 1. / (Dyf/cs2+0.5) # relaxation parameter 
print(Dyf,omega)
# weights
w  =   np.array([4/9., 1/9.,1/9.,1/9.,1/9., 1/36.,1/36.,1/36.,1/36.])

c = [(0,0), (1,0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, 1), (-1, -1), (1, -1)]
opp = [c.index( (-c_[0],-c_[1]) )  for c_ in c]
c = np.array(c)
# numpy version
#opp = [np.where(np.all((c == -c_),axis=-1))[0][0]  for c_ in c]

obst = np.ones((nx,ny)).astype(np.bool)
obst[1:-1,1:-1] = False 


def f_eq(rho,u,c=c,w=w):
    cu = np.tensordot(c,u,axes=[1,0])
    f = rho * (1 + cu/cs2 + (cu**2)/(2*cs2**2) - np.sum((u**2),axis=0)/(2*cs2) )
    return f*w[:,np.newaxis,np.newaxis]

T_init_l = lambda x:(np.exp( -((x[0] - .5)**2+(x[1] - .0)**2)/.050 ))
rho_init = T_init_l([X,Y])

x0,x1 = -2.,2.
y0,y1 = -2.,2.

X,Y = np.mgrid[0:nx,0:ny]
X = x0 + X*(x1-x0)/(nx-1)
Y = y0 + Y*(y1-y0)/(ny-1)

X,Y = np.meshgrid(np.linspace(x0,x1,nx),np.linspace(y0,y1,ny),indexing='ij')

rho_init = T_init_l([X,Y])
rho_init[:] = 1.0

a = .1
b = .4
d = .31
fx = lambda u,v: u-u**3 - v + d
fy = lambda u,v: u-a-b*v

#fx = lambda u,v: 5.1
#fy = lambda u,v: 1.2

u_adv = np.zeros((2,nx,ny))
u_adv[0,:,:] = 1/u0*fx(X,Y)
u_adv[1,:,:] = 1/u0*fy(X,Y)

f = f_eq(rho_init,u_adv,c=c,w=w)

u_t = []
for iteration in range(1550):

    rho = f.sum(axis=0)

    fOut = f - omega * (f-f_eq(rho,u_adv))
    
    for i in range(9):  
        fOut[i,obst] = f[opp[i],obst]

    for i,(k,l) in enumerate(c):
        f[i,:,:] = np.roll(np.roll(fOut[i,:,:],k,axis=0),l,axis=1)
        
    if iteration%10==0:
        u_t.append( rho.copy() )


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets.widgets import IntSlider
from ipywidgets import interact,Layout
style = Layout(width='70%')

f,ax = plt.subplots(figsize=(8,6))
#ax.imshow(T_init_l([X,Y]),origin='lower')
plt_evo = ax.imshow(u_t[0].T,origin='lower',extent=(x0,x1,y0,y1),vmax=12.42,cmap='rainbow')
plt_init = ax.contour(X,Y,T_init_l([X,Y]),colors='r')

ax.set_aspect(1)

@interact(ith=IntSlider(min=0,max=len(u_t)-1,layout=style))
def _(ith):
    plt.title(r'$\int_V \rho(x,y) dxdy =%f$'%np.sum(u_t[ith]))
    plt_evo.set_array(u_t[ith].T)
    

## in a function

In [ ]:
import numpy as np 

def D2Q9_solve_adv_diff(Dyf=0.1,u_adv=(1,1),ic=1.0,Niter=100,time_evo=False):
    """
       numpy with matrix indexing (ij)
    """    
    nx,ny = ic.shape
    cs2 = 1.0/3.0
    omega = 1. / (Dyf/cs2+0.5) # relaxation parameter 
    # weights
    w  =   np.array([4/9., 1/9.,1/9.,1/9.,1/9., 1/36.,1/36.,1/36.,1/36.])

    c = [(0,0), (1,0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, 1), (-1, -1), (1, -1)]
    opp = [c.index( (-c_[0],-c_[1]) )  for c_ in c]
    c = np.array(c)

    obst = np.ones((nx,ny)).astype(np.bool)
    obst[1:-1,1:-1] = False 


    def f_eq(rho,u,c=c,w=w):
        cu = np.tensordot(c,u,axes=[1,0])
        f = rho * (1 + cu/cs2 + (cu**2)/(2*cs2**2) - np.sum((u**2),axis=0)/(2*cs2) )
        return f*w[:,np.newaxis,np.newaxis]

    rho_init = np.empty_like( ic )

    if type(a)==np.ndarray:
        rho_init = ic
    else:
        rho_init[:] = ic

    print(np.sum(rho_init))
    f = f_eq(rho_init,u_adv,c=c,w=w)

    u_t = []
    for iteration in range(Niter):

        rho = f.sum(axis=0)

        fOut = f - omega * (f-f_eq(rho,u_adv))

        for i in range(9):  
            fOut[i,obst] = f[opp[i],obst]

        for i,(k,l) in enumerate(c):
            f[i,:,:] = np.roll(np.roll(fOut[i,:,:],k,axis=0),l,axis=1)

        if (iteration%10==0 and time_evo) or iteration==(Niter-1):
            u_t.append( rho.copy() )
    
    return u_t

In [ ]:
u0 = 11.0
T_init_l = lambda x:(np.exp( -((x[0] - .5)**2+(x[1] - .0)**2)/.050 ))

x0,x1 = -2.,2.
y0,y1 = -2.,2.

nx = 100
ny = 100

X,Y = np.meshgrid(np.linspace(x0,x1,nx),np.linspace(y0,y1,ny),indexing='ij')

ic = T_init_l([X,Y])

a = .1
b = .4
d = .31
fx = lambda u,v: u-u**3 - v + d
fy = lambda u,v: u-a-b*v

u_adv = np.zeros((2,nx,ny))
u_adv[0,:,:] = 1/u0*fx(X,Y)
u_adv[1,:,:] = 1/u0*fy(X,Y)

%time u_t = D2Q9_solve_adv_diff(Dyf=0.01, u_adv=u_adv, ic=ic, Niter=1000, time_evo=True)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets.widgets import IntSlider
from ipywidgets import interact,Layout
style = Layout(width='70%')

f,ax = plt.subplots(figsize=(8,6))
#ax.imshow(T_init_l([X,Y]),origin='lower')
plt_evo = ax.imshow(u_t[0].T,origin='lower',extent=(x0,x1,y0,y1),vmax=.2,cmap='rainbow')
plt_init = ax.contour(X,Y,T_init_l([X,Y]),colors='r')

ax.set_aspect(1)

@interact(ith=IntSlider(min=0,max=len(u_t)-1,layout=style))
def _(ith):
    plt.title(r'$\int_V \rho(x,y) dxdy =%f$'%np.sum(u_t[ith]))
    plt_evo.set_array(u_t[ith].T)
    

In [ ]:
u_t1 = D2Q9_solve_adv_diff(Dyf=0.01, u_adv=u_adv, ic=ic, Niter=1000, time_evo=True)

In [ ]:
u_t2 = D2Q9_solve_adv_diff(Dyf=0.01*0.5, u_adv=0.5*u_adv, ic=ic, Niter=2000, time_evo=True)

In [ ]:
u_t3 = D2Q9_solve_adv_diff(Dyf=0.01/2, u_adv=u_adv[:,::2,::2], ic=ic[::2,::2], Niter=500, time_evo=True)

## Peclet number

$$Pe = \frac{u L}{D}$$



In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets.widgets import IntSlider
from ipywidgets import interact,Layout
style = Layout(width='70%')

f,(ax1,ax2) = plt.subplots(ncols=2,figsize=(8,6))
plt_evo1 = ax1.imshow(u_t1[0].T,origin='lower',extent=(x0,x1,y0,y1),vmax=.2,cmap='rainbow')

plt_evo2 = ax2.imshow(u_t3[0].T,origin='lower',extent=(x0,x1,y0,y1),vmax=.2,cmap='rainbow')

ax.set_aspect(1)

@interact(ith=IntSlider(min=0,max=len(u_t1)-1,layout=style))
def _(ith):
    plt.title(r'$\int_V \rho(x,y) dxdy =%f$'%np.sum(u_t[ith]))
    plt_evo1.set_array(u_t1[ith].T)
    plt_evo2.set_array(u_t3[int(ith/2)].T)

    